In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-10 13:50:26.338549: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 13:50:26.438904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 13:50:26.438920: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 13:50:26.460028: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 13:50:26.978588: W tensorflow/stream_executor/platform/de

In [2]:
name_all=[]
auc_roc=[]
name='all_algorithm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

2022-11-10 13:50:32.048712: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 13:50:32.049362: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:50:32.049434: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:50:32.049491: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-10 13:50:32.049548: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

Epoch 1/100
1/1 [==============================] - 0s 402ms/step - loss: 2.2751 - auc: 0.5425
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.9875 - auc: 0.8239
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 2.0178 - auc: 0.8556
Epoch 4/100
1/1 [==============================] - 0s 27ms/step - loss: 1.9325 - auc: 0.8700
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7631 - auc: 0.8832
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5691 - auc: 0.9021
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4267 - auc: 0.9152
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3772 - auc: 0.9103
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3367 - auc: 0.9083
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2488 - auc: 0.9305
Epoch 11/100
1/1 [==============================] - 0s 40ms/step - l

1/1 [==============================] - 0s 30ms/step - loss: 0.1861 - auc: 0.9955
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1861 - auc: 0.9957
Epoch 90/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1860 - auc: 0.9959
Epoch 91/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1872 - auc: 0.9949
Epoch 92/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1889 - auc: 0.9955
Epoch 93/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1825 - auc: 0.9962
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1853 - auc: 0.9958
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1881 - auc: 0.9951
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1842 - auc: 0.9957
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1833 - auc: 0.9953
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [3]:
name='remove_dmpnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 335ms/step - loss: 1.9240 - auc: 0.5672
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7089 - auc: 0.8099
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7416 - auc: 0.8296
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6466 - auc: 0.8520
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4851 - auc: 0.8674
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.3172 - auc: 0.8833
Epoch 7/100
1/1 [==============================] - 0s 26ms/step - loss: 1.2262 - auc: 0.8900
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1888 - auc: 0.8858
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1513 - auc: 0.8879
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 1.0590 - auc: 0.9198
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2157 - auc: 0.9934
Epoch 89/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2149 - auc: 0.9938
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2081 - auc: 0.9947
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2118 - auc: 0.9942
Epoch 92/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2164 - auc: 0.9935
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2147 - auc: 0.9935
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2119 - auc: 0.9937
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2107 - auc: 0.9941
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2199 - auc: 0.9926
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2113 - auc: 0.9939
Epoch 98/100
1/1 [==============================] - 0s 23ms/step - loss: 

In [4]:
name='remove_lr'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 334ms/step - loss: 3.6453 - auc: 0.2201
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 2.0227 - auc: 0.5110
Epoch 3/100
1/1 [==============================] - 0s 26ms/step - loss: 1.5052 - auc: 0.8449
Epoch 4/100
1/1 [==============================] - 0s 23ms/step - loss: 1.6408 - auc: 0.8649
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.7717 - auc: 0.8774
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.8860 - auc: 0.8710
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8953 - auc: 0.8782
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8800 - auc: 0.8816
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7809 - auc: 0.8846
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6365 - auc: 0.8997
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2573 - auc: 0.9909
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2559 - auc: 0.9915
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2471 - auc: 0.9923
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2515 - auc: 0.9915
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2617 - auc: 0.9891
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2526 - auc: 0.9904
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2530 - auc: 0.9900
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2590 - auc: 0.9890
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2480 - auc: 0.9909
Epoch 97/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2464 - auc: 0.9915
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [5]:
name='remove_fcnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 334ms/step - loss: 1.9697 - auc: 0.7870
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6546 - auc: 0.7838
Epoch 3/100
1/1 [==============================] - 0s 25ms/step - loss: 1.5477 - auc: 0.7517
Epoch 4/100
1/1 [==============================] - 0s 25ms/step - loss: 1.4318 - auc: 0.7893
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2538 - auc: 0.8728
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1678 - auc: 0.9085
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.0867 - auc: 0.9326
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9986 - auc: 0.9539
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9487 - auc: 0.9608
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 0.8745 - auc: 0.9733
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.1773 - auc: 0.9966
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1849 - auc: 0.9948
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1852 - auc: 0.9956
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1819 - auc: 0.9964
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1821 - auc: 0.9960
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1781 - auc: 0.9962
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1782 - auc: 0.9964
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1824 - auc: 0.9956
Epoch 96/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1784 - auc: 0.9959
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1839 - auc: 0.9954
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [6]:
name='remove_xgb'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['fcnn','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 330ms/step - loss: 2.5867 - auc: 0.3002
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6508 - auc: 0.7456
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6093 - auc: 0.8440
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7309 - auc: 0.8571
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.7448 - auc: 0.8719
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7324 - auc: 0.8772
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6311 - auc: 0.8822
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5115 - auc: 0.8888
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3588 - auc: 0.8975
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2196 - auc: 0.9047
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 22ms/step - loss: 0.2568 - auc: 0.9883
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2514 - auc: 0.9904
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2573 - auc: 0.9882
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2579 - auc: 0.9878
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2566 - auc: 0.9884
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2512 - auc: 0.9891
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2534 - auc: 0.9890
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2555 - auc: 0.9881
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2505 - auc: 0.9892
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2547 - auc: 0.9885
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [7]:
de=pd.DataFrame()
de['name']=name_all
de['auc_roc']=auc_roc
de.to_csv('4model_remove_one_model.csv',index=False)
print(de)

            name   auc_roc
0  all_algorithm  0.944734
1   remove_dmpnn  0.942074
2      remove_lr  0.940165
3    remove_fcnn  0.944711
4     remove_xgb  0.940176
